In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.datasets import cifar10


In [ ]:
def create_vgg16_embedding_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    embedding_model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return embedding_model

def preprocess_images(images):
    preprocessed_images = []
    for img in images:
        img_resized = tf.image.resize(img, (224, 224))  # Resize to 224x224
        img_array = img_to_array(img_resized)
        img_array = np.array(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
        preprocessed_images.append(img_array[0])
    return np.array(preprocessed_images)

In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# # Randomly select 10,000 samples from the training set
np.random.seed(42)
num_samples = 10000
random_indices = np.random.choice(x_train.shape[0], num_samples, replace=False)
x_train = x_train[random_indices]
y_train = y_train[random_indices]

np.random.seed(42)
num_samples = 2000
random_indices = np.random.choice(x_test.shape[0], num_samples, replace=False)
x_test = x_test[random_indices]
y_test = y_test[random_indices]


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
embedding_model = create_vgg16_embedding_model()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train_preprocessed = preprocess_images(x_train)
x_train_embeddings = embedding_model.predict(x_train_preprocessed)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1556s 5s/step


In [ ]:
x_train_embeddings.shape

(10000, 512)

In [ ]:
x_train_embeddings[0].shape

(512,)

In [ ]:
x_test_preprocessed = preprocess_images(x_test)
x_test_embeddings = embedding_model.predict(x_test_preprocessed)

63/63 ━━━━━━━━━━━━━━━━━━━━ 325s 5s/step


In [ ]:
x_test_embeddings.shape

(2000, 512)

In [ ]:
x_test_embeddings[0].shape

(512,)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
np.savez('/content/drive/My Drive/Colab Notebooks/DIP_proj/vgg16_embeddings_x_train.npz', embeddings=x_train_embeddings, labels=y_train)
np.savez('/content/drive/My Drive/Colab Notebooks/DIP_proj/vgg16_embeddings_x_test.npz', embeddings=x_test_embeddings, labels=y_test)
